# 20191324 Jiyun Han
## Question
<ol>
<li> Using multiple sequence analysis and phylogenetic tree reconstruction, can you
estimate the propagation path of SARS-CoV-2 in the US? Justify your idea with
the data.
<li> Based on the number of new mutations & the collection time, can you estimate
the “mutation rate”? (how long does it take to make a new mutation?)
<li> Is there any specific gene having higher rate of mutation? Can you explain why?

In [1]:
#Function for read fasta file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [2]:
#1_(1) USA sequences
#from Bio import SeqIO
seq = read_fasta("../data/2019nCoV_genomes.2020_04_14.fa") #Read the given data using parse.
clean_seq = []
count_num = 0
for s_h, s_seq in seq.items():
    if "USA" in s_h:
        clean_seq.append(seq)
        count_num += 1
        #print(s_h)

print(count_num)
#print(clean_seq)

667


In [3]:
#1_(2) Read GCF_009858895.2 prot info. This is reference file!
ref_seq = '../data/GCF_009858895.2_ASM985889v3_protein.faa'
ref_CoV = read_fasta(ref_seq)
for g_h, g_seq in ref_CoV.items():
    print(g_h, g_seq[:25])

YP_009724389.1 orf1ab polyprotein [Wuhan seafood market pneumonia virus] MESLVPGFNEKTHVQLSLPVLQVRD
YP_009724390.1 surface glycoprotein [Wuhan seafood market pneumonia virus] MFVFLVLLPLVSSQCVNLTTRTQLP
YP_009724391.1 ORF3a protein [Wuhan seafood market pneumonia virus] MDLFMRIFTIGTVTLKQGEIKDATP
YP_009724392.1 envelope protein [Wuhan seafood market pneumonia virus] MYSFVSEETGTLIVNSVLLFLAFVV
YP_009724393.1 membrane glycoprotein [Wuhan seafood market pneumonia virus] MADSNGTITVEELKKLLEQWNLVIG
YP_009724394.1 ORF6 protein [Wuhan seafood market pneumonia virus] MFHLVDFQVTIAEILLIIMRTFKVS
YP_009724395.1 ORF7a protein [Wuhan seafood market pneumonia virus] MKIILFLALITLATCELYHYQECVR
YP_009724396.1 ORF8 protein [Wuhan seafood market pneumonia virus] MKFLVFLGIITTVAAFHQECSLQSC
YP_009724397.2 nucleocapsid phosphoprotein [Wuhan seafood market pneumonia virus] MSDNGPQNQRNAPRITFGGPSDSTG
YP_009725255.1 ORF10 protein [Wuhan seafood market pneumonia virus] MGYINVFAFPFTIYSLLLCRMNSRN
YP_009725295.1 orf1a poly

In [5]:
#1_(3) Clean up file
seq = read_fasta("2019nCoV_genomes.2020_04_14.clean.fa") #Read the given data using parse.
jy_clean_seq = {}
count_num = 0
for s_h, s_seq in seq.items():
    if "USA" in s_h:
        if "2020-" in s_h:
            jy_clean_seq.setdefault(s_h, s_seq)
            count_num += 1
        else:
            pass

print(count_num)

#1_(4) Make fasta file
ofile = open("../data/jy_clean_CoV.fa", "w")

for s_h, s_seq in jy_clean_seq.items():
    if '|' in s_h:
        rep_sh = s_h.replace('|', '@')
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
ofile.close()

#1_(5) Using transeq for translation
'''
transeq --sequence jy_clean_CoV.fa --outseq jy_clean_CoV_trans6.fa -frame 6
'''

646


'\ntranseq --sequence jy_clean_CoV.fa --outseq jy_clean_CoV_trans6.fa -frame 6\n'

In [6]:
#Instead of using python, I draw the tree with internet site. My computer can't align with mega & muscle program...
'''
from Bio.Align.Applications import MuscleCommandline

muscle_exe = "/muscle/muscle3.8.31_i86linux64"
cmd_line = MuscleCommandline(muscle_exe, input = "jy_clean_CoV_trans6.fa", out = "jy_CoV.aln", clw = "")

print(cmd_line)

stdout, stderr = cmd_line()
'''

'\nfrom Bio.Align.Applications import MuscleCommandline\n\nmuscle_exe = "/muscle/muscle3.8.31_i86linux64"\ncmd_line = MuscleCommandline(muscle_exe, input = "jy_clean_CoV_trans6.fa", out = "jy_CoV.aln", clw = "")\n\nprint(cmd_line)\n\nstdout, stderr = cmd_line()\n'

In [11]:
#1_(6) Tree
'''
With tree, I read them and think about USA CoV virus propagation.
It explain on word file, that I upload on BlackBoard

with dna data
https://ngphylogeny.fr/data/display/0d71b4225c176834

https://ngphylogeny.fr/workspace/history/a0e758242f7231fa
'''
#1_(7) Propagation
'''
Explained on the BB
'''

'\nExplained on the BB\n'

In [15]:
#2_(1) Get index from GCF and made them to fasta file
'''

Based on GCF_009858895.2_ASM985889v3_genomic.gff
There are proteins that name is
[orf1ab, S, ORF3a, E, M, ORF6, ORF7a, ORF7B, N, ORF8, ORF10]

'''

def read_index(tmp_filename, protein):
    f = open(tmp_filename, 'r')
    for line in f.readlines():
        if 'gene' in line and 'Name={}'.format(protein) in line:
            return int(line.split()[3])-1, int(line.split()[4])-1
        
index_read = '../data/GCF_009858895.2_ASM985889v3_genomic.gff'
#GCF_index = read_index(index_read, 'N')
#print(GCF_index)

ref_seq_gene = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
ref_CoV_gene = read_fasta(ref_seq_gene)

def get_index(prot_name):
    
    GCF_index = read_index(index_read, prot_name)
    i_s = GCF_index[0]
    i_e = GCF_index[1]
    #print(i_s, i_e)
    for g_h, g_seq in ref_CoV_gene.items():
        #print(g_seq)
        seq_prot = g_seq[i_s:i_e]
        #print(seq_prot)
        ofile = open(prot_name, "w")

        lines = ['>', g_h, '\n', seq_prot, '\n']
        ofile.writelines(lines)
        ofile.close()

        

In [16]:
get_index('orf1ab')
get_index('S')
get_index('ORF3a')
get_index('E')
get_index('M')
get_index('ORF6')
get_index('ORF7a')
get_index('ORF7b')
get_index('N')
get_index('ORF8')
get_index('ORF10')

In [14]:
#2_(2) Make them to protein file
'''
transeq --sequence orf1ab --outseq orf1ab_trans6.fa -frame 6
transeq --sequence S --outseq S_trans6.fa -frame 6
transeq --sequence ORF3a --outseq ORF3a_trans6.fa -frame 6
transeq --sequence E --outseq E_trans6.fa -frame 6
transeq --sequence M --outseq M_trans6.fa -frame 6
transeq --sequence ORF6 --outseq ORF6_trans6.fa -frame 6
transeq --sequence ORF7a --outseq ORF7a_trans6.fa -frame 6
transeq --sequence ORF7b --outseq ORF7b_trans6.fa -frame 6
transeq --sequence N --outseq N_trans6.fa -frame 6
transeq --sequence ORF8 --outseq ORF8_trans6.fa -frame 6
transeq --sequence ORF10 --outseq ORF10_trans6.fa -frame 6
'''

'\ntranseq --sequence orf1ab --outseq orf1ab_trans6.fa -frame 6\ntranseq --sequence S --outseq S_trans6.fa -frame 6\ntranseq --sequence ORF3a --outseq ORF3a_trans6.fa -frame 6\ntranseq --sequence E --outseq E_trans6.fa -frame 6\ntranseq --sequence M --outseq M_trans6.fa -frame 6\ntranseq --sequence ORF6 --outseq ORF6_trans6.fa -frame 6\ntranseq --sequence ORF7a --outseq ORF7a_trans6.fa -frame 6\ntranseq --sequence ORF7b --outseq ORF7b_trans6.fa -frame 6\ntranseq --sequence N --outseq N_trans6.fa -frame 6\ntranseq --sequence ORF8 --outseq ORF8_trans6.fa -frame 6\ntranseq --sequence ORF10 --outseq ORF10_trans6.fa -frame 6\n'

In [18]:
#2_(3) Make the own fasta file
orf1 = read_fasta("../data/orf1ab_trans6.fa")
s = read_fasta("../data/S_trans6.fa")
orf3 = read_fasta("../data/ORF3a_trans6.fa")
e = read_fasta("../data/E_trans6.fa")
m = read_fasta("../data/M_trans6.fa")
orf6 = read_fasta("../data/ORF6_trans6.fa")
orf7a = read_fasta("../data/ORF7a_trans6.fa")
orf7b = read_fasta("../data/ORF7b_trans6.fa")
n = read_fasta("../data/N_trans6.fa")
orf8 = read_fasta("../data/ORF8_trans6.fa")
orf10 = read_fasta("../data/ORF10_trans6.fa")

ofile = open("prot_ref", "w")

for s_h, s_seq in orf1.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in s.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in e.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in m.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in orf6.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in orf7a.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in orf7b.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in n.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
for s_h, s_seq in orf8.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)      
for s_h, s_seq in orf10.items():
        lines = ['>', rep_sh, '\n', s_seq, '\n']
        ofile.writelines(lines)
ofile.close()